# Decision Trees and Random Forests example using the wine database
The problem in this base is identify the wine productor based on wine features

In [1]:
import numpy as np  
import matplotlib.pyplot as plt

## Load the dataset and convert it to pandas data frame

In [2]:
from sklearn import datasets
import pandas as pd

data = datasets.load_wine()
# convert sklearn dataset to pandas data frame
wine = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns= data['feature_names'] + ['target'])

wine.info()
wine.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
alcohol                         178 non-null float64
malic_acid                      178 non-null float64
ash                             178 non-null float64
alcalinity_of_ash               178 non-null float64
magnesium                       178 non-null float64
total_phenols                   178 non-null float64
flavanoids                      178 non-null float64
nonflavanoid_phenols            178 non-null float64
proanthocyanins                 178 non-null float64
color_intensity                 178 non-null float64
hue                             178 non-null float64
od280/od315_of_diluted_wines    178 non-null float64
proline                         178 non-null float64
target                          178 non-null float64
dtypes: float64(14)
memory usage: 19.5 KB


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0


## Split the dataset
First into input features (X) and output (y) and then into train and test sets with size 0.8/0.2

In [3]:
from sklearn.model_selection import train_test_split
from pyspark.sql import DataFrame, SparkSession
from pyspark.mllib.regression import LabeledPoint

spark = SparkSession.builder.master("local").appName("wine").getOrCreate()

X_train, X_test = train_test_split(wine, test_size=0.3)

s_df_train = spark.createDataFrame(X_train)
s_df_test = spark.createDataFrame(X_test)
train  = s_df_train.rdd.map(lambda x: LabeledPoint(x[13], x[:13])).collect()
test  = s_df_train.rdd.map(lambda x: LabeledPoint(x[13], x[:13])).collect()

## Create and train the Decision Tree Classifier

In [4]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

model = DecisionTree.trainClassifier(X_train, numClasses=3, categoricalFeaturesInfo={}, impurity='gini', maxDepth=5, maxBins=32)

# model.fit(X_train, y_train)

TypeError: first() takes exactly 2 arguments (1 given)

## Use the model to predict the samples in the test set

In [ ]:
model.fit(X_train,y_train)
predictions = model.predict(X_test)

## Evaluate the model

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

## Display the confusion matrix 

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix

cfm = confusion_matrix(y_test,predictions)
sns.heatmap(cfm, cbar=False, annot=True, cmap="Blues", fmt="d")

## Let's create an image with the decision tree
You'll need some extra libs (IPython and pydot)
* Each OS may require other specific installs.
Mac OS users for example need to install other dependencies. In my case, installing gprofdot (brew install gprof2dot) was enough.

In [ ]:
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 

features = list(wine.columns[:-1])
features

dot_data = StringIO()  
export_graphviz(model, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())  
img = Image(graph[0].create_png())
graph[0].write_png('tree_wine_viz.png')
img

# Random Forests
One tree performed well but lets see if Random Forest can improve the result.
## Create a Random Forest with 100 trees
Evaluate the result and then display the confusion matrix

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train,y_train)
predictions = forest.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
cfm = confusion_matrix(y_test,predictions)
sns.heatmap(cfm, cbar=False, annot=True, cmap="Blues", fmt="d")